In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [ ]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [ ]:
import neural_ode.ODESolvers

In [ ]:
tf.keras.backend.set_floatx('float64')

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        layers.Dense(2, name="layer1"),
    ]
)


In [ ]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, 
                                       solver=neural_ode.ODESolvers.HeunsMethod())

In [ ]:
n_ode.model.variables

In [ ]:
n_ode.pretrain(t_scale=1.0, step_size=0.5, n_epoch = 1)

In [ ]:
# test newtons method

In [ ]:
y0 = tf.constant([[3.0, 4.0]], dtype=tf.float64)
resid = lambda y: y*y-tf.constant([[2.0, 3.0]], dtype=tf.float64)
jac = lambda y: tf.eye(2, dtype=tf.float64)*y*2
neural_ode.ODESolvers.newtons_method(y0, resid, jac)

In [ ]:
sol11

In [ ]:
resid(sol11)

In [ ]:
n_ode.model.variables

In [ ]:
# Generate solution

In [ ]:
#
N_n = int(2)
c = 0.1
k = 4.0
def oscilator(t, y):
    return np.array([y[1], -c*y[1]-k*y[0]])
t_final = 20.0
n_eval = int(501)
t_span = np.array([0.0, t_final])
y0 = np.array([1.0, 0.0])

In [ ]:
sol = solve_ivp(oscilator, t_span, y0, t_eval=np.linspace(0, t_final, num=n_eval))

In [ ]:
plt.plot(sol.t, sol.y[0,:])

In [ ]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )

In [ ]:
sol, _ = n_ode.solver(n_ode.ode_wrap, tf.constant([0.0, 0.1], dtype=tf.float64), tf.expand_dims(y0_tf, axis=0),
                            step_size=0.05, jac = n_ode.grad_inps_wrap)

In [ ]:
plt.plot(sol['t'].numpy(), sol['y'][:,0].numpy())

In [ ]:
sol1 = n_ode.forward_solve(t_target, y_target[0,:])
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
# adjoint method

In [ ]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:])

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.3, -k+1.0], [1.3, -c+0.2]]))
n_ode.model.variables[1].assign(np.array([0.2,0]))

In [ ]:
# fit

In [ ]:
n_epoch = 30
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=10, adjoint_method=False, 
          opt=tf.keras.optimizers.Adam(learning_rate=0.05))

In [ ]:
n_ode.model.variables

In [ ]:
# Check derivatives

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.0, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss, dl_dp = n_ode.usual_method(t_target, y_target)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target)
#dl_dp = a[2:]

In [ ]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target)

In [ ]:
(loss2-loss)/dp

In [ ]:
dl_dp